In [1]:
import tensorflow as tf
tf.__version__

'2.0.0'

# THIS NOTEBOOK IS CURRENTLY BROKEN - use tf.estimator.ipynb instead (using numpy arrays of tf.data.Dataset)

In [11]:
train, _ = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [10]:
iter(dataset).next()[0]

<tf.Tensor: id=29, shape=(), dtype=uint8, numpy=9>

In [111]:
def pre_process(image_tensor, label_tensor):
    #flatten
    image_tensor = tf.reshape(image_tensor,(28**2,))
    
    #tensor to tensordict
    tensor_dict = dict()
    for feature_idx in range(784):
        key = str(feature_idx)
        value = tf.slice(image_tensor,[feature_idx],[1])
        tensor_dict.update({key:value})
        
    #one hot encode labels, cast to int32
    label_tensor = tf.one_hot(label_tensor,10,dtype=tf.int32)
    
    #reshape
    #label_tensor = tf.reshape(label_tensor,[1,10])
    
    return (tensor_dict,label_tensor)

In [52]:
features = []
for feature_idx in range(784):
    features.append(tf.feature_column.numeric_column(str(feature_idx)))

In [115]:
estimator = tf.estimator.DNNClassifier(feature_columns=features, hidden_units = [100], n_classes=10)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/95/7hxgmrh92z91hylbxw647jhr0000gn/T/tmph8hxm8fv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x5d23b38d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [116]:

@tf.function
def get_data():
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset_preprocessed = dataset.map(pre_process)
    return dataset_preprocessed
    

estimator.train(input_fn=get_data, steps=2000)


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: Shape mismatch: The shape of labels (received (10,)) should equal the shape of logits except for the last dimension (received (1, 10)).